<a href="https://colab.research.google.com/github/food-hazard-detection-semeval-2025/food-hazard-detection-semeval-2025.github.io/blob/main/code/The_Food_Hazard_Detection_Challenge_SemEval_2025_The_BERT_Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
%%capture
!pip install torch transformers datasets pandas scikit-learn

In [34]:
# incidents_rest.csv is the dataset besides the trial
# incidents_sample.csv is the trial dataset

In [35]:
# from transformers import BertTokenizer
from transformers import RobertaTokenizer, RobertaForSequenceClassification

import pandas as pd
data = pd.read_csv('incidents_train.csv', index_col=0)
# trial = pd.read_csv('incidents_sample.csv', index_col=0)
data.sample()

,year,month,day,country,title,text,hazard-category,product-category,hazard,product
3114,2018,10,19,ca,Emborg brand Pizza Style Dairy Free Shreds rec...,Food Recall Warning (Allergen) - Emborg brand ...,allergens,prepared dishes and snacks,milk and products thereof,pizza


In [36]:
import pandas as pd
from datasets import Dataset
# from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_scheduler, DataCollatorWithPadding
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW, get_scheduler, DataCollatorWithPadding
from torch.utils.data import DataLoader
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm.auto import tqdm
from sklearn.metrics import classification_report

In [37]:
data.text.str.split().apply(len).describe()

,text
count,5082.000000
mean,353.254034
std,297.662839
min,10.000000
25%,195.000000
50%,306.000000
75%,424.000000
max,8406.000000


In [38]:
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

def tokenize_function(examples):
    return tokenizer(examples['text'], padding=True, truncation=True)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


# Label: `Hazard Category`

* Choose your target

In [39]:
label = 'hazard-category' # change this to: 'product-category', 'hazard', 'product' to alter the ground truth
label_encoder = LabelEncoder()
data['label'] = label_encoder.fit_transform(data[label])

* Data preprocessing

In [40]:
# Split the data into training and testing sets
train_df, test_df = train_test_split(data, test_size=0.2, random_state=42)

# Convert DataFrame to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Apply the tokenizer to the dataset
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Create DataCollator to handle padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True, max_length=16)

# Convert dataset to PyTorch format
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# Create DataLoader objects
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=8, collate_fn=data_collator)
test_dataloader = DataLoader(test_dataset, batch_size=8, collate_fn=data_collator)

Map:   0%|          | 0/4065 [00:00<?, ? examples/s]

Map:   0%|          | 0/1017 [00:00<?, ? examples/s]

* Choose your model

In [41]:
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(data[label].unique()))
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=len(data[label].unique()))
model.to('cuda')  # Move model to GPU if available

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

* Train it

In [42]:
optimizer = AdamW(model.parameters(), lr=5e-5)

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

model.train()

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to('cuda') for k, v in batch.items()}  # Move batch to GPU if available
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/1527 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2888: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


* Assess it

In [43]:
model.eval()
total_predictions = []
with torch.no_grad():
    for batch in test_dataloader:
        batch = {k: v.to('cuda') for k, v in batch.items()}  # Move batch to GPU if available
        outputs = model(**batch)
        predictions = torch.argmax(outputs.logits, dim=-1)
        total_predictions.extend([p.item() for p in predictions])

#print(classification_report(test_df.label, total_predictions))
predicted_labels = label_encoder.inverse_transform(total_predictions)
gold_labels = label_encoder.inverse_transform(test_df.label.values)
print(classification_report(gold_labels, predicted_labels, zero_division=0))

                                precision    recall  f1-score   support

                     allergens       0.95      0.98      0.97       377
                    biological       0.99      0.99      0.99       339
                      chemical       0.88      0.97      0.92        68
food additives and flavourings       0.00      0.00      0.00         5
                foreign bodies       0.96      0.99      0.98       111
                         fraud       0.83      0.71      0.76        68
                     migration       0.00      0.00      0.00         1
          organoleptic aspects       0.78      0.70      0.74        10
                  other hazard       0.70      0.70      0.70        27
              packaging defect       1.00      0.55      0.71        11

                      accuracy                           0.94      1017
                     macro avg       0.71      0.66      0.68      1017
                  weighted avg       0.94      0.94      0.94 

In [44]:
model.save_pretrained("bert_hazard_category")

# Label: `Product Category`

In [45]:
label = 'product-category'
label_encoder = LabelEncoder()
data['label'] = label_encoder.fit_transform(data[label])

# Split the data into training and testing sets
train_df, test_df = train_test_split(data, test_size=0.2, random_state=42)

# Convert DataFrame to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Apply the tokenizer to the dataset
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Create DataCollator to handle padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True, max_length=16)

# Convert dataset to PyTorch format
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# Create DataLoader objects
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=8, collate_fn=data_collator)
test_dataloader = DataLoader(test_dataset, batch_size=8, collate_fn=data_collator)

Map:   0%|          | 0/4065 [00:00<?, ? examples/s]

Map:   0%|          | 0/1017 [00:00<?, ? examples/s]

* Train

In [46]:
# model_product_category = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(data[label].unique()))
model_product_category = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=len(data[label].unique()))
model_product_category.to('cuda')  # Move model to GPU if available

optimizer = AdamW(model_product_category.parameters(), lr=5e-5)
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

model_product_category.train()
progress_bar = tqdm(range(num_training_steps))
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to('cuda') for k, v in batch.items()}  # Move batch to GPU if available
        outputs = model_product_category(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/1527 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2888: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


* Test

In [47]:
model_product_category.eval()
total_predictions = []
with torch.no_grad():
    for batch in test_dataloader:
        batch = {k: v.to('cuda') for k, v in batch.items()}  # Move batch to GPU if available
        outputs = model_product_category(**batch)
        predictions = torch.argmax(outputs.logits, dim=-1)
        total_predictions.extend([p.item() for p in predictions])

#print(classification_report(test_df.label, total_predictions, zero_division=0))
predicted_labels = label_encoder.inverse_transform(total_predictions)
gold_labels = label_encoder.inverse_transform(test_df.label.values)
print(classification_report(gold_labels, predicted_labels, zero_division=0))

                                                   precision    recall  f1-score   support

                              alcoholic beverages       0.75      0.86      0.80         7
                      cereals and bakery products       0.71      0.76      0.74       123
     cocoa and cocoa preparations, coffee and tea       0.73      0.76      0.74        49
                                    confectionery       0.49      0.47      0.48        40
dietetic foods, food supplements, fortified foods       0.69      0.75      0.72        24
                                    fats and oils       1.00      0.50      0.67         4
                                   feed materials       0.00      0.00      0.00         3
                           food contact materials       0.00      0.00      0.00         1
                            fruits and vegetables       0.73      0.74      0.73       112
                                 herbs and spices       0.38      0.50      0.43        1

In [48]:
model_product_category.save_pretrained("bert_product_category")

# Label: `Hazard`

In [49]:
label = 'hazard'
label_encoder = LabelEncoder()
data['label'] = label_encoder.fit_transform(data[label])

# Split the data into training and testing sets
train_df, test_df = train_test_split(data, test_size=0.2, random_state=42)

# Convert DataFrame to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Apply the tokenizer to the dataset
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Create DataCollator to handle padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True, max_length=16)

# Convert dataset to PyTorch format
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# Create DataLoader objects
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=8, collate_fn=data_collator)
test_dataloader = DataLoader(test_dataset, batch_size=8, collate_fn=data_collator)

Map:   0%|          | 0/4065 [00:00<?, ? examples/s]

Map:   0%|          | 0/1017 [00:00<?, ? examples/s]

In [50]:
# model_hazard = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(data[label].unique()))
model_hazard = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=len(data[label].unique()))
model_hazard.to('cuda')  # Move model to GPU if available

optimizer = AdamW(model_hazard.parameters(), lr=5e-5)

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

model_hazard.train()

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to('cuda') for k, v in batch.items()}  # Move batch to GPU if available
        outputs = model_hazard(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/1527 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2888: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


In [51]:
model_hazard.eval()
total_predictions = []
with torch.no_grad():
    for batch in test_dataloader:
        batch = {k: v.to('cuda') for k, v in batch.items()}  # Move batch to GPU if available
        outputs = model_hazard(**batch)
        predictions = torch.argmax(outputs.logits, dim=-1)
        total_predictions.extend([p.item() for p in predictions])

#print(classification_report(test_df.label, total_predictions, zero_division=0))
predicted_labels = label_encoder.inverse_transform(total_predictions)
gold_labels = label_encoder.inverse_transform(test_df.label.values)
print(classification_report(gold_labels, predicted_labels, zero_division=0))

                                                   precision    recall  f1-score   support

                                        Aflatoxin       1.00      0.50      0.67         4
                                  alcohol content       0.00      0.00      0.00         1
                                        alkaloids       0.00      0.00      0.00         2
                                        allergens       0.00      0.00      0.00         4
                                           almond       0.72      0.93      0.81        14
             altered organoleptic characteristics       0.00      0.00      0.00         2
                                        amygdalin       0.00      0.00      0.00         2
                           antibiotics, vet drugs       0.00      0.00      0.00         1
                                    bacillus spp.       1.00      1.00      1.00         1
                             bad smell / off odor       0.00      0.00      0.00         

In [52]:
model_hazard.save_pretrained("bert_hazard")

# Label: `product`

In [53]:
label = 'product'
label_encoder = LabelEncoder()
data['label'] = label_encoder.fit_transform(data[label])

# Split the data into training and testing sets
train_df, test_df = train_test_split(data, test_size=0.2, random_state=42)

# Convert DataFrame to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Apply the tokenizer to the dataset
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Create DataCollator to handle padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True, max_length=16)

# Convert dataset to PyTorch format
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# Create DataLoader objects
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=8, collate_fn=data_collator)
test_dataloader = DataLoader(test_dataset, batch_size=8, collate_fn=data_collator)

Map:   0%|          | 0/4065 [00:00<?, ? examples/s]

Map:   0%|          | 0/1017 [00:00<?, ? examples/s]

In [54]:
# model_product = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(data[label].unique()))
model_product = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=len(data[label].unique()))
model_product.to('cuda')  # Move model to GPU if available

optimizer = AdamW(model_product.parameters(), lr=5e-5)
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
model_product.train()
progress_bar = tqdm(range(num_training_steps))
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to('cuda') for k, v in batch.items()}  # Move batch to GPU if available
        outputs = model_product(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/1527 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2888: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


In [55]:
model_product.eval()
total_predictions = []
with torch.no_grad():
    for batch in test_dataloader:
        batch = {k: v.to('cuda') for k, v in batch.items()}  # Move batch to GPU if available
        outputs = model_product(**batch)
        predictions = torch.argmax(outputs.logits, dim=-1)
        total_predictions.extend([p.item() for p in predictions])

#print(classification_report(test_df.label, total_predictions, zero_division=0))
predicted_labels = label_encoder.inverse_transform(total_predictions)
gold_labels = label_encoder.inverse_transform(test_df.label.values)
print(classification_report(gold_labels, predicted_labels, zero_division=0))

                                                   precision    recall  f1-score   support

                           Catfishes (freshwater)       0.00      0.00      0.00         5
                            Fishes not identified       0.00      0.00      0.00         6
                         Not classified pork meat       0.00      0.00      0.00         3
                       Pangas catfishes (generic)       0.00      0.00      0.00         1
              Precooked cooked pork meat products       0.00      0.00      0.00         1
                                    Veggie Burger       0.00      0.00      0.00         2
                               after dinner mints       0.00      0.00      0.00         1
                                            algae       0.00      0.00      0.00         3
                            all purpose seasoning       0.00      0.00      0.00         1
                                   almond kernels       0.00      0.00      0.00         

In [56]:
import sys
sys.exit("Stopping notebook execution here.")

# ---------------------------------------------
model_product.save_pretrained("bert_product")
tokenizer.save_pretrained("bert_tokenizer")

SystemExit: Stopping notebook execution here.

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
!zip bert_baseline.zip bert_*

# Loading a trained baseline

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

def predict(texts, model_path, tokenizer_path="bert_tokenizer"):
    # Load the saved tokenizer
    tokenizer = BertTokenizer.from_pretrained(tokenizer_path)

    # Load the saved model
    model = BertForSequenceClassification.from_pretrained(model_path)

    # Move model to GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Tokenize the input texts
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")

    # Move inputs to the same device as the model
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Set the model to evaluation mode
    model.eval()

    # Make predictions
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

    return predictions

In [ ]:
from sklearn.metrics import f1_score

def compute_score(hazards_true, products_true, hazards_pred, products_pred):
  # compute f1 for hazards:
  f1_hazards = f1_score(
    hazards_true,
    hazards_pred,
    average='macro'
  )

  # compute f1 for products:
  f1_products = f1_score(
    products_true[hazards_pred == hazards_true],
    products_pred[hazards_pred == hazards_true],
    average='macro'
  )

  return (f1_hazards + f1_products) / 2.

### Sub-Task 1:

In [ ]:
predictions = predict(trial.title.to_list(), "bert_hazard_category")

# Decode predictions back to string labels
label_encoder = LabelEncoder()
label_encoder.fit(data['hazard-category'])

trial['bert-hazard-category'] = predict(trial.title.to_list(), "bert_hazard_category")
print(classification_report(trial['hazard-category'], trial['bert-hazard-category'], zero_division=0))

In [ ]:
predictions = predict(trial.title.to_list(), "bert_product_category")

# Decode predictions back to string labels
label_encoder = LabelEncoder()
label_encoder.fit(data['product-category'])

trial['bert-product-category'] = predict(trial.title.to_list(), "bert_product_category")
print(classification_report(trial['product-category'], trial['bert-product-category'], zero_division=0))

In [ ]:
print('Score Sub-Task 1:', compute_score(trial['hazard-category'], trial['product-category'], trial['bert-hazard-category'], trial['bert-product-category']))

### Sub-Task 2:

In [ ]:
predictions = predict(trial.title.to_list(), "bert_hazard")

# Decode predictions back to string labels
label_encoder = LabelEncoder()
label_encoder.fit(data['hazard'])

trial['bert-hazard'] = predict(trial.title.to_list(), "bert_hazard")
print(classification_report(trial['hazard'], trial['bert-hazard'], zero_division=0))

In [ ]:
predictions = predict(trial.title.to_list(), "bert_product")

# Decode predictions back to string labels
label_encoder = LabelEncoder()
label_encoder.fit(data['product'])

trial['bert-product'] = predict(trial.title.to_list(), "bert_product")
print(classification_report(trial['product'], trial['bert-product'], zero_division=0))

In [ ]:
print('Score Sub-Task 2:', compute_score(trial['hazard'], trial['product'], trial['bert-hazard'], trial['bert-product']))